In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT REPLACE(SPLIT(current_user(), '@')[0], '.', '_');

In [0]:
USE IDENTIFIER("fhir_workshop." || schema_use);

In [0]:
SELECT current_catalog(), current_schema();

In [0]:
DROP TABLE IF EXISTS fhir_bundle_metadata;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_bundle_metadata 
COMMENT 'Original FHIR Bundle Metadata'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,file_metadata
FROM STREAM (fhir_bronze_variant);

In [0]:
DROP TABLE IF EXISTS fhir_resources;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_resources 
COMMENT 'Exploded FHIR Resources'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  bundle_uuid
  ,CAST(entry.value:fullUrl AS STRING) as fullUrl
  ,CAST(entry.value:resource.resourceType AS STRING) as resourceType
  ,resource.*
FROM 
  STREAM(fhir_bronze_variant)
  ,LATERAL variant_explode(fhir:entry) as entry
  ,LATERAL variant_explode(entry.value:resource) as resource

In [0]:
SELECT * FROM fhir_resources;

In [0]:
DROP TABLE IF EXISTS fhir_resource_schemas;

In [0]:
CREATE OR REFRESH STREAMING TABLE fhir_resource_schemas 
COMMENT 'Schemas of inferred from FHIR Resource Variant Data Types'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT
  resourceType
  ,key as resource
  ,REPLACE(schema_of_variant_agg(value), 'OBJECT', 'STRUCT') as schema_of_variant
FROM STREAM(fhir_resources)
GROUP BY 
  resourceType
  ,key

In [0]:
select * from fhir_resource_schemas;